# 7 Visualise activity in a video.

We have extracted all the features we plan to use. Overlaying these on the video was useful.
But watching annotated videos is inefficient and not always informative.. 

To help with understanding we build a few tools that let's see at a glance what happens over time.

# 7.1 Plot movements over time. 

In each frame let's find the `centre of gravity` for each person (the average of all the high-confidence marker points). This is handy for time series visualisation. For example plotting the cog.x for each person over time shows how they move closer and further from each other. 

In [ ]:
import os
import utils
import calcs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ultralytics

In [ ]:
videos_in = r"..\LookitLaughter.test"
data_out = r"..\data\1_interim"
videos_out = r"..\data\2_final"

#a couple of files for testing
VIDEO_FILE  = os.path.join(videos_in, "2UWdXP.joke1.rep2.take1.Peekaboo.mp4")
VIDEO_FILE2 = os.path.join(videos_in, "2UWdXP.joke2.rep1.take1.NomNomNom.mp4")
AUDIO_FILE = os.path.join(data_out, "2UWdXP.joke1.rep2.take1.Peekaboo.wav")
SPEECH_FILE = os.path.join(data_out, "2UWdXP.joke1.rep2.take1.Peekaboo.json")

testset = [VIDEO_FILE, VIDEO_FILE2] 

processedvideos = utils.getprocessedvideos(data_out)
processedvideos.head()

Let's get the keypoint data and calculate

In [ ]:
keypoints = utils.readKeyPointsFromCSV(processedvideos,VIDEO_FILE)


In [ ]:
#this bit of pandas magic calculates average x and y for all the rows.

keypoints[["cogx","cogy"]] = keypoints.apply(lambda row: calcs.rowcogs(row.iloc[8:59]), axis=1, result_type='expand')

In [ ]:
child = keypoints[keypoints["person"]=="child"]
adult = keypoints[keypoints["person"]=="adult"]

#a plot of child's centre of gravity frame by frame
childplot = plt.plot(child["frame"], child["cogx"], c="red", alpha=0.5)
## add line of adult's centre of gravity
adultplot = plt.plot(adult["frame"], adult["cogx"], c="blue", alpha=0.5)
#add legend
plt.legend(['child', 'adult'], loc='upper left')

In [ ]:
#out of curiousity, let's plot the location of the adult and child's nose

childnoseplot = plt.plot(child["frame"], child["nose.x"], c="red", alpha=0.5)
adultnoseplot = plt.plot(adult["frame"], adult["nose.x"], c="blue", alpha=0.5)
plt.legend(['child', 'adult'], loc='upper left')

# 7.2 TODO Use Voxel51 and PytorchVideo for examining videos

Voxel51 seems to be a useful tool for looking at training data (and trained predictions).

Let's start with the minimal implementation. Just viewing videos.

https://docs.voxel51.com/user_guide/dataset_creation/index.html



In [ ]:
import fiftyone as fo

In [ ]:
fo.delete_datasets("*")

In [ ]:



# Create a dataset from a directory of videos
dataset = fo.Dataset.from_videos_dir("../LookitLaughter.test")
dataset.ensure_frames()

dataset.name = 'LookitLaughter.test'


dataset.add_sample_field("JokeType", fo.StringField, description="What joke is being told?")
dataset.add_sample_field("HowFunny", fo.StringField, description="How funny is the joke?")
dataset.add_sample_field("LaughYesNo",  fo.BooleanField, description="Did the child laugh?")




Now let's see if we can add our metadata classifications. Recalling that each video demos one joke type `[Peekaboo,TearingPaper,NomNomNom,ThatsNotAHat,ThatsNotACat]` and has rating of how funny the baby found it `[Not Funny, Slightly Funny, Funny, Extremely Funny]` and whether they laughed `[Yes, No]`.


In [ ]:
# add the joke type, how funny and laugh yes/no for each sample in the dataset
for sample in dataset:
    videoname = sample.filepath.split("\\")[-1]
    row = processedvideos[processedvideos["VideoID"]==videoname]
    sample["VideoID"]  = row["VideoID"].values[0]
    sample["JokeType"]  = row["JokeType"].values[0]
    sample["HowFunny"]  = row["HowFunny"].values[0]
    sample["LaughYesNo"]  = (row["LaughYesNo"].values[0] == "Yes")
    sample.save()

Let's add the frame by frame annotations directly onto the videos inside fiftyone

In [ ]:
def xyxy2ltwh(bbox):
    """Convert [x1 y1 x2 y2] box format to [x y w h] format."""
    if isinstance(bbox, np.ndarray):
        bbox = bbox.tolist()
    return [bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]]

In [ ]:
#Let's start with people bounding boxes


for sample in dataset:
    #retrieve people bounding boxes from the keypoints file
    keypoints = utils.readKeyPointsFromCSV(processedvideos,sample.filepath,normed= True)    

    for index, row in keypoints.iterrows():
        framenumber = row["frame"] + 1
        person = row["person"]
        bbox = [row["bbox.x1"], row["bbox.y1"], row["bbox.x2"], row["bbox.y2"]]
        bbox51 = xyxy2ltwh(bbox)
        if sample.frames[framenumber]:
            frame = sample.frames[framenumber]
        else:
            frame = fo.Frame()
        frame[person] = fo.Detection(label=person, bounding_box=bbox51)
        sample.frames[framenumber] = frame
        frame[person + "KeyPoints"] =  
        fo.KeypointSkeleton()
        sample.save()



In [ ]:
session = fo.launch_app(dataset)